In [6]:
import keras
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Conv2D=================================================================================================================================================================================================================================================================, Input, GlobalAveragePooling2D,Dense, Activation, Flatten, BatchNormalization
# from keras.applications.mobilenet import relu6, DepthwiseConv2D
from keras.layers import DepthwiseConv2D
from keras import Model
from keras import optimizers
# from keras import activations

In [8]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()
train_x = train_x.reshape((60000,28,28,1)) / 255.0
test_x = test_x.reshape((10000,28,28,1)) / 255.0

train_y = keras.utils.to_categorical(train_y, num_classes = 10)
test_y = keras.utils.to_categorical(test_y, num_classes = 10)
print (train_y.shape)
print (test_y.shape)

11493376/11490434 [==============================] - 25s 2us/step
(60000, 10)
(10000, 10)


#### DepthwiseConv2D
```py
class DepthwiseConv2D(Conv2D):
    """Depthwise separable 2D convolution.
    Depthwise Separable convolutions consist in performing
    just the first step in a depthwise spatial convolution
    (which acts on each input channel separately).
    It does not perform the pointwise convolution (second step).
    The `depth_multiplier` argument controls how many
    output channels are generated per input channel in the depthwise step.
    # Arguments
        kernel_size: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
            Can be a single integer to specify the same value for
            all spatial dimensions.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
            Specifying any stride value != 1 is incompatible with specifying
            any `dilation_rate` value != 1.
        padding: one of `"valid"` or `"same"` (case-insensitive).
        depth_multiplier: The number of depthwise convolution output channels
            for each input channel.
            The total number of depthwise convolution output
            channels will be equal to `filterss_in * depth_multiplier`.
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        depthwise_initializer: Initializer for the depthwise kernel matrix
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        depthwise_regularizer: Regularizer function applied to
            the depthwise kernel matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        depthwise_constraint: Constraint function applied to
            the depthwise kernel matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
    # Input shape
        4D tensor with shape:
        `(batch, channels, rows, cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(batch, rows, cols, channels)` if data_format='channels_last'.
    # Output shape
        4D tensor with shape:
        `(batch, filters, new_rows, new_cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(batch, new_rows, new_cols, filters)` if data_format='channels_last'.
        `rows` and `cols` values might have changed due to padding.
    """
```

In [9]:
def MobileNetV1(input_shape, k):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(inputs)
    x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
    x = Conv2D(filters=64, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu',)(x)
    x = Conv2D(filters=128, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
    x = Conv2D(filters=128, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
    x = Conv2D(filters=256, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
    x = Conv2D(filters=256, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
    x = Conv2D(filters=512, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
    x = Conv2D(filters=512, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
    x = Conv2D(filters=1024, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
    x = Conv2D(filters=1024, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(k, activation='softmax')(x)
    model = Model(inputs, x)
    return model

In [10]:
def NaiveConvNet(input_shape, k):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = Conv2D(filters=128, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
    x = Conv2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = Conv2D(filters=512, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
    x = Conv2D(filters=1024, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
    x = Conv2D(filters=1024, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(k, activation='softmax')(x)
    model = Model(inputs, x)
    return model

In [11]:
MobileNet_v1_model = MobileNetV1((224,224,3), 1000)
MobileNet_v1_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 112, 112, 32)      320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 112, 112, 64)      2112      
_________________________________________________________________
depthwise_conv2d_2 (Depthwis (None, 112, 112, 64)      640       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 128)     8320      
_________________________________________________________________
depthwise_conv2d_3 (Depthwis (None, 56, 56, 128)       1280      
__________

In [12]:
NaiveConvNet_model = NaiveConvNet((224,224,3), 1000)
NaiveConvNet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 112, 32)      896       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 56, 56, 128)       147584    
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 256)       590080    
__________

In [13]:
def Simple_MobileNetV1(input_shape, k):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(inputs)
    x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
    x = Conv2D(filters=64, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), padding='same', activation='relu',)(x)
    x = Conv2D(filters=128, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
    x = Conv2D(filters=128, kernel_size=(1,1),padding='same', activation='relu')(x)
    x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
#     x = Conv2D(filters=256, kernel_size=(1,1),padding='same', activation='relu')(x)
#     x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
#     x = Conv2D(filters=256, kernel_size=(1,1),padding='same', activation='relu')(x)
# #     x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
# #     x = Conv2D(filters=512, kernel_size=(1,1),padding='same', activation='relu')(x)
# #     x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
# #     x = Conv2D(filters=512, kernel_size=(1,1),padding='same', activation='relu')(x)
# #     x = DepthwiseConv2D(kernel_size=(3,3), strides=(2,2),padding='same', activation='relu',)(x)
# #     x = Conv2D(filters=1024, kernel_size=(1,1),padding='same', activation='relu')(x)
# #     x = DepthwiseConv2D(kernel_size=(3,3),padding='same', activation='relu',)(x)
    x = Conv2D(filters=128, kernel_size=(1,1), padding='same', activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(k, activation='softmax')(x)
    model = Model(inputs, x)
    return model

In [15]:
Simple_MobileNetV1_model = Simple_MobileNetV1((28,28,1), 10)
Simple_MobileNetV1_model.summary()
Simple_MobileNetV1_model.compile(optimizer=optimizers.RMSprop(0.001), loss='categorical_crossentropy', metrics=['acc'])
Simple_MobileNetV1_history = Simple_MobileNetV1_model.fit(train_x, train_y, epochs=20, batch_size=8)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
depthwise_conv2d_14 (Depthwi (None, 14, 14, 32)        320       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 14, 14, 64)        2112      
_________________________________________________________________
depthwise_conv2d_15 (Depthwi (None, 14, 14, 64)        640       
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 14, 14, 128)       8320      
_________________________________________________________________
depthwise_conv2d_16 (Depthwi (None, 7, 7, 128)         1280      
__________

In [16]:
Simple_MobileNetV1_model.evaluate(test_x, test_y)

10000/10000 [==============================] - 1s 66us/step


[0.0815806967404671, 0.9776]

In [17]:
def Simple_NaiveConvNet(input_shape, k):
    inputs = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same', activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = Conv2D(filters=128, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
#     x = Conv2D(filters=256, kernel_size=(3,3),padding='same', activation='relu')(x)
#     x = Conv2D(filters=256, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
#     x = Conv2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(x)
#     x = Conv2D(filters=512, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
#     x = Conv2D(filters=1024, kernel_size=(3,3), strides=(2,2),padding='same', activation='relu')(x)
#     x = Conv2D(filters=1024, kernel_size=(3,3),padding='same', activation='relu')(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(k, activation='softmax')(x)
    model = Model(inputs, x)
    return model

In [18]:
Simple_NaiveConvNet_model = Simple_NaiveConvNet((28,28,1), 10)
Simple_NaiveConvNet_model.summary()
Simple_NaiveConvNet_model.compile(optimizer=optimizers.RMSprop(0.001), loss='categorical_crossentropy', metrics=['acc'])
Simple_NaiveConvNet_history = Simple_NaiveConvNet_model.fit(train_x, train_y, epochs=50, batch_size=8)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 7, 7, 128)         147584    
_________________________________________________________________
global_average_pooling2d_6 ( (None, 128)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 128)               512       
__________